In [10]:
pip install groq python-dotenv

In [16]:
from groq import Groq

# 🔑 Paste your Groq API Key here
client = Groq(api_key="")

# ✅ Use Active Model From Your Account
BASE_MODEL = "llama-3.1-8b-instant"

# -----------------------------------
# 🎯 Expert Configurations
# -----------------------------------
MODEL_CONFIG = {
    "technical": {
        "system_prompt": """You are a senior software engineer.
Provide precise, code-focused, and technically rigorous answers.
If debugging, clearly explain the root cause and provide corrected code snippets.
Be concise and accurate."""
    },

    "billing": {
        "system_prompt": """You are a customer support billing specialist.
Be empathetic and professional.
Explain charges, refund policies, and subscription details clearly.
Reassure the customer and provide next steps."""
    },

    "general": {
        "system_prompt": """You are a helpful customer support assistant.
Answer politely and clearly.
Handle casual or general queries."""
    }
}

# -----------------------------------
# 🚦 Router
# -----------------------------------
def route_prompt(user_input: str) -> str:

    lower_input = user_input.lower()

    # ✅ Rule-based Tool Override (Deterministic)
    if "bitcoin" in lower_input and ("price" in lower_input or "current" in lower_input):
        return "tool"

    # Otherwise use LLM router
    routing_prompt = f"""
Classify the following customer query into one of these categories:
technical, billing, general

Return ONLY one word from the list above.

Query:
{user_input}
"""

    response = client.chat.completions.create(
        model=BASE_MODEL,
        messages=[
            {"role": "system", "content": "You are a strict intent classifier."},
            {"role": "user", "content": routing_prompt}
        ],
        temperature=0
    )

    category = response.choices[0].message.content.strip().lower()

    if category not in ["technical", "billing", "general"]:
        return "general"

    return category


# -----------------------------------
# 🧰 Tool Expert
# -----------------------------------
def get_bitcoin_price():
    return "$62,450"  # Mock value


# -----------------------------------
# 🎼 Orchestrator
# -----------------------------------
def process_request(user_input: str):

    category = route_prompt(user_input)
    print(f"\n[Router Decision] → {category}")

    if category == "tool":
        if "bitcoin" in user_input.lower():
            return f"The current price of Bitcoin is {get_bitcoin_price()}."
        else:
            return "Requested tool is not available."

    system_prompt = MODEL_CONFIG[category]["system_prompt"]

    response = client.chat.completions.create(
        model=BASE_MODEL,
        messages=[
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": user_input}
        ],
        temperature=0.7
    )

    return response.choices[0].message.content


# -----------------------------------
# 🖥️ Main Loop
# -----------------------------------
if __name__ == "__main__":

    print("🧠 Smart Customer Support Router (MoE)")
    print("Type 'exit' to quit.\n")

    while True:
        user_query = input("Enter your query: ")

        if user_query.lower() == "exit":
            print("Goodbye 👋")
            break

        result = process_request(user_query)

        print("\nResponse:")
        print(result)
        print("\n----------------------------------\n")

🧠 Smart Customer Support Router (MoE)
Type 'exit' to quit.

Enter your query: My python script throws IndexError

[Router Decision] → technical

Response:
IndexError in Python typically occurs when you're trying to access an element in a list or a string that doesn't exist. Here are some common causes:

1. **Out-of-range indexing**: You're trying to access an index that's greater than or equal to the length of the list or string.

Example:

```python
my_list = [1, 2, 3]
print(my_list[3])  # IndexError: list index out of range
```

2. **Negative indexing**: You're trying to access an index that's less than 0.

Example:

```python
my_list = [1, 2, 3]
print(my_list[-1])  # This is okay, but print(my_list[-4]) would raise an IndexError
```

3. **List modification while iterating**: You're modifying a list while iterating over it, which can cause unexpected behavior and IndexError.

Example:

```python
my_list = [1, 2, 3]
for i in range(len(my_list)):
    if my_list[i] == 2:
        my_list

In [12]:
print(client.models.list())

ModelListResponse(data=[Model(id='groq/compound-mini', created=1756949707, object='model', owned_by='Groq', active=True, context_window=131072, public_apps=None, max_completion_tokens=8192), Model(id='openai/gpt-oss-20b', created=1754407957, object='model', owned_by='OpenAI', active=True, context_window=131072, public_apps=None, max_completion_tokens=65536), Model(id='llama-3.1-8b-instant', created=1693721698, object='model', owned_by='Meta', active=True, context_window=131072, public_apps=None, max_completion_tokens=131072), Model(id='whisper-large-v3', created=1693721698, object='model', owned_by='OpenAI', active=True, context_window=448, public_apps=None, max_completion_tokens=448), Model(id='moonshotai/kimi-k2-instruct', created=1752435491, object='model', owned_by='Moonshot AI', active=True, context_window=131072, public_apps=None, max_completion_tokens=16384), Model(id='canopylabs/orpheus-v1-english', created=1766186316, object='model', owned_by='Canopy Labs', active=True, contex